In [1]:

pip install cefpython3


Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector
import pandas as pd
import pandas as pd
from tkinter import ttk
import os
from tkhtmlview import HTMLLabel
from tkinter import filedialog

In [11]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector
import pandas as pd

# ---------------- Database Connection ----------------
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="@tanishq123",
        database="employee_tracker"
    )


# ---------------- Styling ----------------
BG_COLOR = "#f0f8ff"
BTN_COLOR = "#0077b6"
DANGER_COLOR = "#d62828"
TITLE_COLOR = "#023e8a"

# ---------------- Theme Management ----------------
LIGHT_THEME = {
    "BG_COLOR": "#f0f8ff",
    "BTN_COLOR": "#0077b6",
    "DANGER_COLOR": "#d62828",
    "TITLE_COLOR": "#023e8a",
    "FG_COLOR": "black"
}

DARK_THEME = {
    "BG_COLOR": "#222831",
    "BTN_COLOR": "#00adb5",
    "DANGER_COLOR": "#ff4c4c",
    "TITLE_COLOR": "#eeeeee",
    "FG_COLOR": "white"
}

current_theme = LIGHT_THEME.copy()

def apply_theme(window):
    """Apply current theme to a given window and its widgets"""
    window.configure(bg=current_theme["BG_COLOR"])
    for widget in window.winfo_children():
        try:
            widget.configure(bg=current_theme["BG_COLOR"], fg=current_theme["FG_COLOR"])
        except:
            pass

def style_button(btn, color=None):
    if color is None:
        color = current_theme["BTN_COLOR"]
    btn.configure(
        bg=color,
        fg="white",
        font=("Helvetica", 10, "bold"),
        relief="raised",
        bd=2,
        cursor="hand2",
        activebackground=color
    )

def toggle_theme():
    global current_theme
    current_theme = DARK_THEME.copy() if current_theme == LIGHT_THEME else LIGHT_THEME.copy()
    apply_theme(root)
    for w in root.winfo_children():
        try:
            w.configure(bg=current_theme["BG_COLOR"], fg=current_theme["FG_COLOR"])
        except:
            pass

# ---------------- Utility Functions ----------------
def create_centered_window(title, width, height):
    win = tk.Toplevel(root)
    win.title(title)
    screen_width = win.winfo_screenwidth()
    screen_height = win.winfo_screenheight()
    x = int((screen_width / 2) - (width / 2))
    y = int((screen_height / 2) - (height / 2))
    win.geometry(f"{width}x{height}+{x}+{y}")
    apply_theme(win)
    return win


# ---------------- Login ----------------
def login():
    user_id = entry_employee_id.get()
    password = entry_password.get()
    login_type = login_var.get()
    

    # HR Login
    if login_type == "HR" and user_id == "Ac101" and password == "6114":
        open_hr_dashboard(user_id, "HR Admin")
        return

    # Employee Login
    if login_type == "Employee":
        try:
            con = connect_db()
            cursor = con.cursor()
            cursor.execute("SELECT * FROM employee WHERE employee_id=%s AND password=%s", (user_id, password))
            result = cursor.fetchone()
            con.close()

            if result:
                employee_name = result[1]
                open_employee_dashboard(user_id, employee_name)
            else:
                messagebox.showerror("Login Failed", "Invalid ID or Password")
        except Exception as e:
            messagebox.showerror("Error", f"Database error:\n{e}")

# ---------------- Employee Dashboard ----------------
def open_employee_dashboard(employee_id, employee_name):
    dashboard = create_centered_window(f"Employee Dashboard - {employee_name}", 500, 650)
    tk.Label(dashboard, text=f"Welcome, {employee_name}", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    sections = {
        "Employee Profile": lambda: open_profile_section(employee_id),
        "Dashboard Overview": lambda: embed_power_bi_dashboard(employee_id),
        "Performance Report": lambda: open_performance(employee_id),
        "Leave Management": lambda: open_leave_management(employee_id, employee_name),
        "Attendance & Timesheet": lambda: open_attendance(employee_id),
        "Work History": lambda: open_work_history(employee_id),
        "Profile Settings": lambda: open_profile_settings(employee_id),
        "Feedback & Support": lambda: open_feedback_support(employee_id),
    }

    for text, command in sections.items():
        btn = tk.Button(dashboard, text=text, command=command, width=30, height=2)
        style_button(btn)
        btn.pack(pady=5)

# ---------------- HR Dashboard ----------------
def open_hr_dashboard(hr_id, hr_name):
    dashboard = create_centered_window(f"HR Dashboard - {hr_name}", 600, 600)
    tk.Label(dashboard, text=f"Welcome HR, {hr_name}", font=("Helvetica", 14, "bold"),
             bg=current_theme["BG_COLOR"], fg=current_theme["FG_COLOR"]).pack(pady=10)

    # ---- Dropdown to select Employee ----
    tk.Label(dashboard, text="Select Employee to View Data:", 
             bg=current_theme["BG_COLOR"], fg=current_theme["FG_COLOR"]).pack(pady=5)

    emp_var = tk.StringVar()
    emp_combo = ttk.Combobox(dashboard, textvariable=emp_var, width=30)
    emp_combo.pack(pady=5)

    # Fetch employees from DB
    try:
        con = connect_db()
        cursor = con.cursor()
        cursor.execute("SELECT employee_id, employee_name FROM employee")
        employees = cursor.fetchall()
        con.close()
        emp_combo['values'] = [f"{eid} - {ename}" for eid, ename in employees]
    except Exception as e:
        messagebox.showerror("Error", f"Database error: {e}")

    # Button to View Employee Details
    def view_selected_employee():
        selected = emp_var.get()
        if not selected:
            messagebox.showwarning("Warning", "Please select an employee")
            return
        emp_id = selected.split(" - ")[0]
        open_employee_dashboard(emp_id, selected.split(" - ")[1])

    btn_view_emp = tk.Button(dashboard, text="View Selected Employee Dashboard", 
                             command=view_selected_employee, width=35)
    style_button(btn_view_emp)
    btn_view_emp.pack(pady=10)

    # ---- Existing HR features ----
    sections = {
        "View All Employees": view_all_employees,
        "Add Employee": add_employee_window,
        "Delete Employee": delete_employee_window,
        "All Attendance": view_all_attendance,
        "Search Employee Attendance": view_employee_attendance_by_id,
        "All Leave Requests": view_all_leaves,
    }

    for text, command in sections.items():
        btn = tk.Button(dashboard, text=text, command=command, width=35, height=2)
        style_button(btn)
        btn.pack(pady=5)

# ---------------- HR Functions ----------------
def view_all_employees():
    win = create_centered_window("All Employees", 600, 400)
    tk.Label(win, text="All Employee Records", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    tree = ttk.Treeview(win, columns=("ID", "Name", "Phone", "Email"), show="headings")
    for col in ("ID", "Name", "Phone", "Email"):
        tree.heading(col, text=col)
        tree.column(col, width=120)
    tree.pack(fill="both", expand=True)

    try:
        con = connect_db()
        cursor = con.cursor()
        cursor.execute("SELECT employee_id, employee_name, phone, email FROM employee")
        for row in cursor.fetchall():
            tree.insert("", "end", values=row)
        con.close()
    except Exception as e:
        messagebox.showerror("Error", f"Database error:\n{e}")

def add_employee_window():
    win = create_centered_window("Add Employee", 400, 350)
    tk.Label(win, text="Add New Employee", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    labels = ["Employee ID", "Name", "Phone", "Email", "Password"]
    entries = []
    for lbl in labels:
        tk.Label(win, text=lbl, bg=BG_COLOR).pack()
        e = tk.Entry(win)
        e.pack(pady=5)
        entries.append(e)

    def save_employee():
        data = [e.get() for e in entries]
        if all(data):
            try:
                con = connect_db()
                cursor = con.cursor()
                cursor.execute("INSERT INTO employee VALUES (%s, %s, %s, %s, %s)", tuple(data))
                con.commit()
                con.close()
                messagebox.showinfo("Success", "Employee added successfully!")
                win.destroy()
            except Exception as e:
                messagebox.showerror("Error", f"Database error:\n{e}")
        else:
            messagebox.showwarning("Warning", "Please fill all fields")

    btn = tk.Button(win, text="Add Employee", command=save_employee)
    style_button(btn)
    btn.pack(pady=10)

def delete_employee_window():
    win = create_centered_window("Delete Employee", 350, 200)
    tk.Label(win, text="Enter Employee ID to Delete:", font=("Helvetica", 12), bg=BG_COLOR).pack(pady=10)
    entry_id = tk.Entry(win)
    entry_id.pack(pady=5)

    def delete_emp():
        emp_id = entry_id.get()
        if emp_id:
            try:
                con = connect_db()
                cursor = con.cursor()
                cursor.execute("DELETE FROM employee WHERE employee_id=%s", (emp_id,))
                con.commit()
                con.close()
                messagebox.showinfo("Deleted", "Employee removed successfully")
                win.destroy()
            except Exception as e:
                messagebox.showerror("Error", f"Database error:\n{e}")
        else:
            messagebox.showwarning("Warning", "Please enter an ID")

    btn = tk.Button(win, text="Delete Employee", command=delete_emp)
    style_button(btn, color=DANGER_COLOR)
    btn.pack(pady=10)

def view_all_attendance():
    win = create_centered_window("All Attendance", 700, 400)
    tk.Label(win, text="All Employees Attendance (Excel)", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    try:
        df = pd.read_excel("Metadata - Copy.xlsx")
        tree = ttk.Treeview(win, columns=list(df.columns), show="headings", height=10)
        for col in df.columns:
            tree.heading(col, text=col)
            tree.column(col, width=100)
        for _, row in df.iterrows():
            tree.insert("", "end", values=list(row))
        tree.pack(fill="both", expand=True)
    except Exception as e:
        messagebox.showerror("Error", f"Could not load Excel:\n{e}")
def view_employee_attendance_by_id():
    win = create_centered_window("Search Employee Attendance", 700, 500)
    tk.Label(win, text="Enter Employee ID:", font=("Helvetica", 12), bg=BG_COLOR).pack(pady=5)
    entry_id = tk.Entry(win)
    entry_id.pack(pady=5)

    result_frame = tk.Frame(win, bg=BG_COLOR)
    result_frame.pack(pady=10, fill="both", expand=True)

    def search():
        emp_id = entry_id.get().strip()
        for widget in result_frame.winfo_children():
            widget.destroy()
        if not emp_id:
            messagebox.showwarning("Warning", "Please enter an employee ID")
            return
        try:
            df = pd.read_excel("Metadata - Copy.xlsx")
            df['employee_id'] = df['employee_id'].astype(str).str.replace('_','',regex=False)
            emp_df = df[df['employee_id'] == emp_id]
            if emp_df.empty:
                tk.Label(result_frame, text="No records found.", bg=BG_COLOR, fg="red").pack()
                return
            cols = ['Date','Day','Time_In','Time_Out','Total_Hours','Workstatus']
            tree = ttk.Treeview(result_frame, columns=cols, show='headings', height=10)
            for col in cols:
                tree.heading(col, text=col)
                tree.column(col, width=100)
            for _, row in emp_df.iterrows():
                tree.insert("", "end", values=[row['Date'],row['Day'],row['Time_In'],row['Time_Out'],row['Total_Hours'],row['Workstatus']])
            tree.pack(fill='both', expand=True)
        except Exception as e:
            messagebox.showerror("Error", f"Could not load Excel data:\n{e}")

    btn_search = tk.Button(win, text="Search", command=search)
    style_button(btn_search)
    btn_search.pack(pady=5)

def view_all_leaves():
    win = create_centered_window("All Leave Requests", 650, 400)
    tk.Label(win, text="All Leave Requests (Approve/Reject)", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    tree = ttk.Treeview(win, columns=("EmpID", "Name", "Start", "End", "Reason", "Status"), show="headings")
    for col in ("EmpID", "Name", "Start", "End", "Reason", "Status"):
        tree.heading(col, text=col)
        tree.column(col, width=100)
    tree.pack(fill="both", expand=True)

    # Load leave requests from DB
    def load_leaves():
        for row in tree.get_children():
            tree.delete(row)
        con = connect_db()
        cursor = con.cursor()
        cursor.execute("SELECT employee_id, employee_name, start_date, end_date, reason, status FROM leave_requests")
        for row in cursor.fetchall():
            tree.insert("", "end", values=row)
        con.close()

    # Update leave status
    def update_leave_status(new_status):
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Select a leave request first!")
            return

        # Extract selected row details
        emp_id = tree.item(selected[0])['values'][0]
        start_date = tree.item(selected[0])['values'][2]

        try:
            con = connect_db()
            cursor = con.cursor()
            cursor.execute("UPDATE leave_requests SET status=%s WHERE employee_id=%s AND start_date=%s",
                           (new_status, emp_id, start_date))
            con.commit()
            con.close()
            messagebox.showinfo("Success", f"Leave {new_status} successfully!")
            load_leaves()
        except Exception as e:
            messagebox.showerror("Error", f"Database error:\n{e}")

    # Buttons for Approve / Reject
    btn_frame = tk.Frame(win, bg=BG_COLOR)
    btn_frame.pack(pady=10)
    tk.Button(btn_frame, text="Approve", command=lambda: update_leave_status('Approved'),
              bg="#2a9d8f", fg="white", width=15).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Reject", command=lambda: update_leave_status('Rejected'),
              bg="#e63946", fg="white", width=15).pack(side="left", padx=10)

    load_leaves()

# ---------------- Employee Functions ----------------
def open_profile_section(employee_id):
    win = create_centered_window("Employee Profile", 400, 300)
    tk.Label(win, text="Employee Profile", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    try:
        con = connect_db()
        cursor = con.cursor()
        cursor.execute("SELECT employee_id, employee_name, phone, email FROM employee WHERE employee_id=%s", (employee_id,))
        data = cursor.fetchone()
        con.close()

        if data:
            labels = ["ID","Name","Phone","Email"]
            for i,val in enumerate(data):
                tk.Label(win, text=f"{labels[i]}: {val}", font=("Helvetica", 12), bg=BG_COLOR, anchor="w").pack(fill="x", padx=20, pady=2)
        else:
            tk.Label(win, text="Employee not found.", fg="red", bg=BG_COLOR).pack(pady=10)
    except Exception as e:
        messagebox.showerror("Error", f"Database error:\n{e}")

# ---------------- Embedded Dashboards ----------------
def embed_power_bi_dashboard(employee_id):
    win = create_centered_window("Power BI Dashboard", 900, 600)
    tk.Label(win, text="Embedded Power BI Dashboard",
             font=("Helvetica", 14, "bold"),
             bg=current_theme["BG_COLOR"], fg=current_theme["FG_COLOR"]).pack(pady=10)

    # Replace with your real Power BI embed link
    power_bi_url = "https://app.powerbi.com/view?r=YOUR_EMBED_LINK"
    html_content = f"""
    <iframe width="850" height="500" src="{power_bi_url}" frameborder="0" allowFullScreen="true"></iframe>
    """
    html_label = HTMLLabel(win, html=html_content, width=900, height=600)
    html_label.pack(pady=10)

def embed_excel_dashboard():
    win = create_centered_window("Excel Dashboard", 900, 600)
    tk.Label(win, text="Embedded Excel Dashboard",
             font=("Helvetica", 14, "bold"),
             bg=current_theme["BG_COLOR"], fg=current_theme["FG_COLOR"]).pack(pady=10)

    file_path = filedialog.askopenfilename(
        title="Select Excel Dashboard",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    if not file_path:
        return

    try:
        df = pd.read_excel(file_path)
        html_table = df.to_html(index=False)
        html_label = HTMLLabel(win, html=html_table, width=900, height=600)
        html_label.pack(pady=10)
    except Exception as e:
        messagebox.showerror("Error", f"Could not load Excel file:\n{e}")

def open_performance(employee_id):
    win = create_centered_window("Performance Report", 400, 600)
    tk.Label(win, text="Daily Performance Report", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    fields = ["Report Date (YYYY-MM-DD)", "Total Calls", "Visits Count", "New Admissions", "NPC", "Follow Up Calls", "Total Demos", "New Calls"]
    entries = {}

    for field in fields:
        tk.Label(win, text=field, font=("Helvetica", 10), bg=BG_COLOR).pack(pady=2)
        e = tk.Entry(win, width=30)
        e.pack(pady=2)
        entries[field] = e

    def submit_report():
        try:
            con = connect_db()
            cursor = con.cursor()
            data = (
                employee_id,
                entries["Report Date (YYYY-MM-DD)"].get(),
                int(entries["Total Calls"].get()),
                int(entries["Visits Count"].get()),
                int(entries["New Admissions"].get()),
                int(entries["NPC"].get()),
                int(entries["Follow Up Calls"].get()),
                int(entries["Total Demos"].get()),
                int(entries["New Calls"].get())
            )
            cursor.execute("""INSERT INTO performance_reports
                (employee_id, report_date, total_calls, visits_count, new_admissions_count,
                 npc, follow_up_calls, total_demos, new_calls)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)""", data)
            con.commit()
            con.close()
            messagebox.showinfo("Success", "Performance report submitted.")
            win.destroy()
        except ValueError:
            messagebox.showerror("Input Error", "Enter numeric values where required.")
        except Exception as e:
            messagebox.showerror("Database Error", str(e))

    btn = tk.Button(win, text="Submit Report", command=submit_report)
    style_button(btn)
    btn.pack(pady=20)

def open_leave_management(employee_id, employee_name):
    win = create_centered_window("Leave Management", 500, 450)
    tk.Label(win, text="Apply for Leave", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    tk.Label(win, text="Start Date (YYYY-MM-DD):", bg=BG_COLOR).pack()
    entry_start = tk.Entry(win)
    entry_start.pack()

    tk.Label(win, text="End Date (YYYY-MM-DD):", bg=BG_COLOR).pack()
    entry_end = tk.Entry(win)
    entry_end.pack()

    tk.Label(win, text="Reason:", bg=BG_COLOR).pack()
    text_reason = tk.Text(win, height=4, width=40)
    text_reason.pack()

    def submit_leave():
        start = entry_start.get()
        end = entry_end.get()
        reason = text_reason.get("1.0", tk.END).strip()
        try:
            con = connect_db()
            cursor = con.cursor()
            cursor.execute(
                "INSERT INTO leave_requests (employee_id, employee_name, start_date, end_date, reason, status) VALUES (%s,%s,%s,%s,%s,'Pending')",
                (employee_id, employee_name, start, end, reason)
            )
            con.commit()
            con.close()
            messagebox.showinfo("Success", "Leave request submitted.")
            win.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Database error:\n{e}")

    btn = tk.Button(win, text="Submit", command=submit_leave)
    style_button(btn)
    btn.pack(pady=10)

def open_attendance(employee_id):
    win = create_centered_window("Attendance & Timesheet", 700, 500)
    tk.Label(win, text="Attendance & Timesheet", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    try:
        df = pd.read_excel("Metadata - Copy.xlsx")
        df['employee_id'] = df['employee_id'].astype(str).str.replace('_', '', regex=False)
        emp_df = df[df['employee_id'] == str(employee_id)]

        full_day_count = (emp_df['Workstatus'] == "Full Day").sum()
        half_day_count = (emp_df['Workstatus'] == "Half Day").sum()
        leave_count = (emp_df['Workstatus'] == "Leave").sum()
        holiday_count = (emp_df['Workstatus'] == "Paid Holiday").sum()
        present_count = full_day_count + half_day_count

        stats_frame = tk.Frame(win, bg=BG_COLOR)
        stats_frame.pack(pady=10)

        def stat_button(text, count):
            btn = tk.Button(stats_frame, text=f"{text}: {count}", width=15)
            style_button(btn)
            return btn

        stat_button("Present", present_count).grid(row=0, column=0, padx=5)
        stat_button("Leaves", leave_count).grid(row=0, column=1, padx=5)
        stat_button("Holidays", holiday_count).grid(row=0, column=2, padx=5)
        stat_button("Half Days", half_day_count).grid(row=0, column=3, padx=5)

        table_frame = tk.Frame(win)
        table_frame.pack(pady=10, fill="both", expand=True)

        cols = ['Date', 'Day', 'Time_In', 'Time_Out', 'Total_Hours', 'Workstatus']
        tree = ttk.Treeview(table_frame, columns=cols, show='headings', height=10)
        for col in cols:
            tree.heading(col, text=col)
            tree.column(col, width=100)

        for _, row in emp_df.iterrows():
            tree.insert("", "end", values=[row['Date'], row['Day'], row['Time_In'], row['Time_Out'], row['Total_Hours'], row['Workstatus']])

        vsb = ttk.Scrollbar(table_frame, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=vsb.set)
        vsb.pack(side='right', fill='y')
        tree.pack(side='left', fill='both', expand=True)
    except Exception as e:
        messagebox.showerror("Error", f"Could not load Excel data:\n{e}")

def open_work_history(employee_id):
    win = create_centered_window("Work History", 500, 300)
    tk.Label(win, text="Work History (Placeholder)", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)
    tk.Label(win, text="Connect this to SQL / Excel to show projects or work logs.", bg=BG_COLOR).pack()

def open_profile_settings(employee_id):
    win = create_centered_window("Profile Settings", 400, 300)
    tk.Label(win, text="Update Profile Settings", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    tk.Label(win, text="New Phone:", bg=BG_COLOR).pack()
    entry_phone = tk.Entry(win)
    entry_phone.pack()

    tk.Label(win, text="New Email:", bg=BG_COLOR).pack()
    entry_email = tk.Entry(win)
    entry_email.pack()

    tk.Label(win, text="New Password:", bg=BG_COLOR).pack()
    entry_password = tk.Entry(win, show="*")
    entry_password.pack()

    def update_profile():
        phone = entry_phone.get()
        email = entry_email.get()
        password = entry_password.get()
        try:
            con = connect_db()
            cursor = con.cursor()
            cursor.execute("UPDATE employee SET phone=%s, email=%s, password=%s WHERE employee_id=%s",
                           (phone, email, password, employee_id))
            con.commit()
            con.close()
            messagebox.showinfo("Success", "Profile updated successfully!")
            win.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Database error:\n{e}")

    btn = tk.Button(win, text="Update", command=update_profile)
    style_button(btn)
    btn.pack(pady=10)

def open_feedback_support(employee_id):
    win = create_centered_window("Feedback & Support", 400, 300)
    tk.Label(win, text="Submit Feedback / Support Request", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    text_feedback = tk.Text(win, height=6, width=40)
    text_feedback.pack(pady=10)

    def submit_feedback():
        feedback = text_feedback.get("1.0", tk.END).strip()
        try:
            con = connect_db()
            cursor = con.cursor()
            cursor.execute("INSERT INTO feedback (employee_id, feedback) VALUES (%s,%s)", (employee_id, feedback))
            con.commit()
            con.close()
            messagebox.showinfo("Success", "Feedback submitted. Thank you!")
            win.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Database error:\n{e}")

    btn = tk.Button(win, text="Submit Feedback", command=submit_feedback)
    style_button(btn)
    btn.pack(pady=10)

# ---------------- Forgot Password ----------------
def forgot_password():
    win = create_centered_window("Forgot Password", 400, 300)
    tk.Label(win, text="Recover / Reset Password", font=("Helvetica", 14, "bold"), bg=BG_COLOR).pack(pady=10)

    # Input Fields
    tk.Label(win, text="Enter Employee ID:", bg=BG_COLOR).pack()
    entry_forgot_id = tk.Entry(win)
    entry_forgot_id.pack(pady=5)

    tk.Label(win, text="Enter Registered Email:", bg=BG_COLOR).pack()
    entry_forgot_email = tk.Entry(win)
    entry_forgot_email.pack(pady=5)

    tk.Label(win, text="Enter New Password:", bg=BG_COLOR).pack()
    entry_new_password = tk.Entry(win, show="*")
    entry_new_password.pack(pady=5)

    def reset_password():
        emp_id = entry_forgot_id.get().strip()
        email = entry_forgot_email.get().strip()
        new_password = entry_new_password.get().strip()

        if not emp_id or not email or not new_password:
            messagebox.showwarning("Warning", "Please fill all fields")
            return

        try:
            con = connect_db()
            cursor = con.cursor()
            
            # Check if employee exists with given ID and Email
            cursor.execute("SELECT * FROM employee WHERE employee_id=%s AND email=%s", (emp_id, email))
            user = cursor.fetchone()

            if user:
                # Update the password
                cursor.execute("UPDATE employee SET password=%s WHERE employee_id=%s", (new_password, emp_id))
                con.commit()
                con.close()
                messagebox.showinfo("Success", "Password has been reset successfully!")
                win.destroy()
            else:
                messagebox.showerror("Error", "No account found with given ID and Email.")
                con.close()

        except Exception as e:
            messagebox.showerror("Error", f"Database error:\n{e}")

    btn_reset = tk.Button(win, text="Reset Password", command=reset_password)
    style_button(btn_reset, color="#2a9d8f")
    btn_reset.pack(pady=15)


# ---------------- Main Login Window ----------------
root = tk.Tk()
root.title("Employee & HR Login")
root.geometry("420x380")
root.configure(bg=BG_COLOR)
root.resizable(False, False)

tk.Label(root, text="Employee Tracker Login", font=("Helvetica", 16, "bold"), bg=BG_COLOR, fg=TITLE_COLOR).pack(pady=15)

frame = tk.Frame(root, bg=BG_COLOR)
frame.pack(pady=10)

tk.Label(frame, text="ID", font=("Helvetica", 12), bg=BG_COLOR).grid(row=0, column=0, padx=10, pady=5)
entry_employee_id = tk.Entry(frame, font=("Helvetica", 12))
entry_employee_id.grid(row=0, column=1, padx=10, pady=5)

tk.Label(frame, text="Password", font=("Helvetica", 12), bg=BG_COLOR).grid(row=1, column=0, padx=10, pady=5)
entry_password = tk.Entry(frame, font=("Helvetica", 12), show="*")
entry_password.grid(row=1, column=1, padx=10, pady=5)

login_var = tk.StringVar(value="Employee")
tk.Radiobutton(root, text="Employee Login", variable=login_var, value="Employee", bg=BG_COLOR).pack()
tk.Radiobutton(root, text="HR Login", variable=login_var, value="HR", bg=BG_COLOR).pack()

btn_login = tk.Button(root, text="Login", command=login)
style_button(btn_login)
btn_login.pack(pady=10)

btn_forgot = tk.Button(root, text="Forgot Password?", command=forgot_password)
style_button(btn_forgot, color="#2a9d8f")
btn_forgot.pack(pady=5)

root.mainloop()


In [4]:
os.getcwd()

'D:\\MyJupyterFiles\\Employer tracker data\\kinter'